In [91]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [98]:
def load_model():
    ns = ['type', 'x', 'y', 'z']
    joints = pd.read_csv('joints.obj', sep=' ', names=ns)
    # we will sort by 'x' axis as all points are located on the x axis
    axis_sort = 'x'
    df = joints.sort_values(by=[axis_sort])
    # arms and elbows are first and last values after sort

    dfJointsVertexesRowCount = len(joints)

    joints = joints.sort_values(by=[axis_sort])

    idxWrist1 = 0
    idxWrist2 = dfJointsVertexesRowCount - idxWrist1 - 1

    idxElbow1 = 1
    idxElbow2 = dfJointsVertexesRowCount - idxElbow1 - 1

    idxShoulder1 = 2
    idxShoulder2 = dfJointsVertexesRowCount - idxShoulder1 - 1

    wrist1 = joints.iloc[idxWrist1]
    wrist2 = joints.iloc[idxWrist2]

    elbow1 = joints.iloc[idxElbow1]
    elbow2 = joints.iloc[idxElbow2]

    shoulder1 = joints.iloc[idxShoulder1]

    shoulder2 = joints.iloc[idxShoulder2]

    # we have 2 vectors which show us the direction for the arms:
    vector_l = np.asarray([wrist1.x - elbow1.x, wrist1.y - elbow1.y, wrist1.z - elbow1.z])
    vector_r = np.asarray([wrist2.x - elbow2.x, wrist2.y - elbow2.y, wrist2.z - elbow2.z])

    # read file with model
    dfModel = pd.read_csv('model.obj', sep=' ', names=['type', 'x', 'y', 'z'])

    # vertexes
    verts = dfModel[dfModel.type == 'v']

    # faces
    faces = dfModel[dfModel.type == 'f']

    faces.loc[:, 'x'] = faces.x.astype(int)
    faces.loc[:, 'y'] = faces.y.astype(int)
    faces.loc[:, 'z'] = faces.z.astype(int)
    return verts, faces, shoulder1, shoulder2, elbow1, elbow2, vector_l, vector_r


def find_vertexes(verts, faces, shoulder1, shoulder2, elbow1, elbow2, vector_l, vector_r):
    dfModelVertexesRowCount = len(verts)
    points = verts.values[:, 1:]
    # create cosine_similarity vector and projection_vector for each point
    # for each point compute distance to left elbow
    vector_to_left = points - elbow1.values[1:]
    distance_to_left = (vector_to_left[:, 0] ** 2 + vector_to_left[:, 1] ** 2 + vector_to_left[:, 2] ** 2) ** 0.5
    # for each point compute distance to right elbow
    vector_to_right = points - elbow2.values[1:]
    distance_to_right = (vector_to_right[:, 0] ** 2 + vector_to_right[:, 1] ** 2 + vector_to_right[:, 2] ** 2) ** 0.5
    # for each point compute similarity with left arm-vector
    cosin_left = cosine_similarity(vector_to_left, vector_l.reshape(1, -1))
    # for each point compute similarity with right arm-vector
    cosin_right = cosine_similarity(vector_to_right, vector_r.reshape(1, -1))
    # for each point take projection to left perpendicular
    projection_l = distance_to_left.reshape(-1, 1) * (1 - cosin_left ** 2) ** 0.5
    # for each point take projection to right perpendicular
    projection_r = distance_to_right.reshape(-1, 1) * (1 - cosin_right ** 2) ** 0.5

    # create column for each data
    verts['projectl'] = projection_l
    verts['distl'] = distance_to_left
    verts['cosl'] = cosin_left
    verts['projectr'] = projection_r
    verts['distr'] = distance_to_right
    verts['cosr'] = cosin_right
    # we check two criterias: left arm and right arm cuts
    verts.loc[:, 'badl'] = 0
    verts.loc[:, 'badr'] = 0
    # criteria 1: for left arm
    criteria_1 = (verts['projectl'] <= 0.1) & (verts['cosl'] >= 0)
    verts.loc[criteria_1, 'badl'] = 1
    # criteria 2: for right arm
    criteria_2 = (verts['projectr'] <= 0.1) & (verts['cosr'] >= 0)
    verts.loc[criteria_2, 'badr'] = 1

    x = np.linspace(1, 6449, 6449)
    # create a column for numeration
    verts.loc[:, 'i'] = x

    # --------------left arm ---------------------------------
    # for each flat we have 3 points: 1 if this point should be deleted; 0 otherwise
    faces.loc[:, '1'] = 0
    faces.loc[:, '2'] = 0
    faces.loc[:, '3'] = 0

    # we have all points for left arm that we want to cut - bad points
    left_points_to_delete = set(verts.loc[verts['badl'] == 1, 'i'])

    faces.loc[faces['x'].isin(left_points_to_delete), '1'] = 1
    faces.loc[faces['y'].isin(left_points_to_delete), '2'] = 1
    faces.loc[faces['z'].isin(left_points_to_delete), '3'] = 1

    # remove flats with only bad points from left side

    faces = faces[~((faces['1'] == 1) & (faces['2'] == 1) & (faces['3'] == 1))]

    # take flats with mixed points: some of them should be projected
    flats_with_mix_points = faces[~((faces['1'] == 0) & (faces['2'] == 0) & (faces['3'] == 0))]

    # project points in the intersection
    bad_points1 = flats_with_mix_points.loc[flats_with_mix_points['1'] == 1, 'x']
    bad_points2 = flats_with_mix_points.loc[flats_with_mix_points['2'] == 1, 'y']
    bad_points3 = flats_with_mix_points.loc[flats_with_mix_points['3'] == 1, 'z']
    left_points_to_project = set(bad_points1) | set(bad_points2) | set(bad_points3)

    # left vector_l should be normalized for projection
    dist_l = (vector_l[0] ** 2 + vector_l[1] ** 2 + vector_l[2] ** 2) ** 0.5
    norm_vector_l = [vector_l[0] / dist_l, vector_l[1] / dist_l, vector_l[2] / dist_l]

    mask = verts['i'].isin(left_points_to_project)
    verts.loc[mask, 'x'] = verts.loc[mask, 'x'] - verts.loc[mask, 'cosl'] * \
                           verts.loc[mask, 'distl'] * norm_vector_l[0]
    verts.loc[mask, 'y'] = verts.loc[mask, 'y'] - verts.loc[mask, 'cosl'] * \
                           verts.loc[mask, 'distl'] * norm_vector_l[1]
    verts.loc[mask, 'z'] = verts.loc[mask, 'z'] - verts.loc[mask, 'cosl'] * \
                                     verts.loc[mask, 'distl'] * norm_vector_l[2]

    # remove bad points
    mask1 = verts['i'].isin(left_points_to_delete) & (~(verts['i'].isin(left_points_to_project)))
    verts.loc[mask1, 'x'] = shoulder1.x
    verts.loc[mask1, 'y'] = shoulder1.y
    verts.loc[mask1, 'z'] = shoulder1.z

    # -----------------right arm -------------------------
    # set values for bad verteces as 0

    faces.loc[:, '1'] = 0
    faces.loc[:, '2'] = 0
    faces.loc[:, '3'] = 0
    right_points_to_delete = set(verts.loc[verts['badr'] == 1, 'i'])

    faces.loc[faces['x'].isin(right_points_to_delete), '1'] = 1
    faces.loc[faces['y'].isin(right_points_to_delete), '2'] = 1
    faces.loc[faces['z'].isin(right_points_to_delete), '3'] = 1

    # remove flats with only bad points from right arm
    faces = faces[~((faces['1'] == 1) & (faces['2'] == 1) & (faces['3'] == 1))]

    # take flats with intersected points only
    flats_with_mix_points = faces[~((faces['1'] == 0) & (faces['2'] == 0) & (faces['3'] == 0))]

    # project points in the intersection
    bad_points1 = flats_with_mix_points.loc[flats_with_mix_points['1'] == 1, 'x']
    bad_points2 = flats_with_mix_points.loc[flats_with_mix_points['2'] == 1, 'y']
    bad_points3 = flats_with_mix_points.loc[flats_with_mix_points['3'] == 1, 'z']
    right_points_to_project = set(bad_points1) | set(bad_points2) | set(bad_points3)

    dist_r = (vector_r[0] ** 2 + vector_r[1] ** 2 + vector_r[2] ** 2) ** 0.5
    norm_vector_r = [vector_r[0] / dist_r, vector_r[1] / dist_r, vector_r[2] / dist_r]

    mask = verts['i'].isin(right_points_to_project)
    verts.loc[mask, 'x'] = verts.loc[mask, 'x'] - verts.loc[mask, 'cosr'] * \
                           verts.loc[mask, 'distr'] * norm_vector_r[0]
    verts.loc[mask, 'y'] = verts.loc[mask, 'y'] - verts.loc[mask, 'cosr'] * \
                           verts.loc[mask, 'distr'] * norm_vector_r[1]
    verts.loc[mask, 'z'] = verts.loc[mask, 'z'] - verts.loc[mask, 'cosr'] * \
                           verts.loc[mask, 'distr'] * norm_vector_r[2]

    # remove bad points
    mask1 = verts['i'].isin(right_points_to_delete) & (~(verts['i'].isin(right_points_to_project)))
    verts.loc[mask1, 'x'] = shoulder2.x
    verts.loc[mask1, 'y'] = shoulder2.y
    verts.loc[mask1, 'z'] = shoulder2.z
    verts['v'] = 'v'
    faces['f'] = 'f'
    # return only data for points
    return faces.loc[:, ['f', 'x', 'y', 'z']], verts.loc[:, ['v', 'x', 'y', 'z']]


def save_model(verts, faces):
    verts.to_csv('model_new.obj', sep=' ', header=False, index=False, float_format='%.6f')
    faces.to_csv('model_new.obj', sep=' ', header=False, index=False, mode='a')


def plot_model(verts):
    pass


def __main__():
    verts, faces, shoulder1, shoulder2, elbow1, elbow2, vector_l, vector_r = load_model()
    faces, verts = find_vertexes(verts, faces, shoulder1, shoulder2, elbow1, elbow2,
                                                  vector_l, vector_r)
    save_model(verts, faces)
    plot_model(verts)

% time __main__()

/home/yustiks/anaconda3/envs/env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 861 ms, sys: 23.7 ms, total: 884 ms
Wall time: 684 ms
